In [1]:
import pandas as pd 
import numpy as np 
from functools import partial # to reduce df memory consumption by applying to_numeric
import warnings
warnings.filterwarnings('ignore') 

In [2]:
data_loc = '/Users/aditi_khullar/Documents/Datascience@Berkeley/W207/FinalProject/data/'

In [3]:
#Read Aisles data
aisles = pd.read_csv(data_loc + 'aisles.csv')
print('Total aisles: {}'.format(aisles.shape[0]))
aisles.head()

Total aisles: 134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
# Read Departments data
departments = pd.read_csv(data_loc + 'departments.csv')
print('Total departments: {}'.format(departments.shape[0]))
departments.head()

Total departments: 21


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
# Read Products data
products = pd.read_csv(data_loc + 'products.csv')
print('Total products: {}'.format(products.shape[0]))
products.head()

Total products: 49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [24]:
# Joining aisles, departments and products
goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')
goods.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,2,All-Seasons Salt,104,13,pantry,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,beverages,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen,frozen meals
4,5,Green Chile Anytime Sauce,5,13,pantry,marinades meat preparation


###### Train Dataset

In [25]:
# Read Train dataset, using efficient dtypes to save space in memory
op_train = pd.read_csv(data_loc + 'order_products__train.csv', dtype={'order_id': np.int32, 'product_id': np.int32, 'add_to_cart_order': np.int16, 'reordered': np.int8})
print('Total ordered products(train): {}'.format(op_train.shape[0]))
op_train.head()

Total ordered products(train): 1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


###### Prior Dataset

In [26]:
# Read prior dataset
op_prior = pd.read_csv(data_loc + 'order_products__prior.csv', dtype={'order_id': np.int32, 'product_id': np.int32,'add_to_cart_order': np.int16, 
'reordered': np.int8})
print('Total ordered products(prior): {}'.format(op_prior.shape[0]))
op_prior.head()

Total ordered products(prior): 32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


###### Test Dataset

In [27]:
# Test dataset
test = pd.read_csv(data_loc + 'sample_submission.csv')
print('Total orders(test): {}'.format(test.shape[0]))
test.head()

Total orders(test): 75000


,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [28]:
# Read orders dataset
orders = pd.read_csv(data_loc + 'orders.csv', dtype={'order_id': np.int32,'user_id': np.int32,'order_number': np.int32, 'order_dow': np.int8, 'order_hour_of_day': np.int8, 'days_since_prior_order': np.float16})
print('Total orders: {}'.format(orders.shape[0]))
orders.head()

Total orders: 3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [29]:
# Merge train and prior together 
# split df indexes into parts
indexes = np.linspace(0, len(op_prior), num=10, dtype=np.int32)

# First merge train and orders
order_details = pd.merge(left=op_train, right=orders, how='left', on='order_id').apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))

# Add order hierarchy
order_details = pd.merge(left=order_details, right=goods[['product_id', 'aisle_id','department_id']].apply(partial(pd.to_numeric, errors='ignore', 
downcast='integer')),how='left',on='product_id')
del op_train
order_details.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,1,49302,1,1,112108,train,4,4,10,9,120,16
1,1,11109,2,1,112108,train,4,4,10,9,108,16
2,1,10246,3,0,112108,train,4,4,10,9,83,4
3,1,49683,4,0,112108,train,4,4,10,9,83,4
4,1,43633,5,1,112108,train,4,4,10,9,95,15


In [30]:
%%time
# Update by small portions
for i in range(len(indexes)-1):
    order_details = pd.concat(
        [   
            order_details,
            pd.merge(left=pd.merge(
                            left=op_prior.loc[indexes[i]:indexes[i+1], :],
                            right=goods[['product_id', 
                                         'aisle_id', 
                                         'department_id' ]].apply(partial(pd.to_numeric, 
                                                                          errors='ignore', 
                                                                          downcast='integer')),
                            how='left',
                            on='product_id'
                            ),
                     right=orders, 
                     how='left', 
                     on='order_id'
                )
        ]
    )
        
print('Datafame length: {}'.format(order_details.shape[0]))
print('Memory consumption: {:.2f} Mb'.format(sum(order_details.memory_usage(index=True, 
                                                                         deep=True) / 2**20)))

# Keep test orders from orders data
test_orders = orders[orders.eval_set == 'test']

# delete (redundant now) dataframes
del op_prior, orders

Datafame length: 33819114
Memory consumption: 3096.23 Mb
CPU times: user 25.8 s, sys: 12.4 s, total: 38.2 s
Wall time: 39.5 s


In [31]:
test_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


###### Customer will take all reordered

In [32]:
%%time
# baseline submission
test_history = order_details[(order_details.user_id.isin(test_orders.user_id)) 
                             & (order_details.reordered == 1)]\
.groupby('user_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()
test_history.columns = ['user_id', 'products']

test_history = pd.merge(left=test_history, 
                        right=test_orders, 
                        how='right', 
                        on='user_id')[['order_id', 'products']]

test_history.to_csv('baseline_takeAllReordered.csv', encoding='utf-8', index=False)


CPU times: user 13.6 s, sys: 936 ms, total: 14.6 s
Wall time: 14.8 s
